# Indeed.com scraping with Selenium

In [1]:
# import packages
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
from selenium import webdriver
import time
from datetime import datetime

- Delete the job title and location in the url and replace it with curly braces. Create a base_url that you'll use to create whatever job position or location that you want.

In [2]:
base_url = 'https://nl.indeed.com/jobs?q={}&l={}&start='

- Create a function called get_url, with the arguments of position and location. Then move the base_url variable inside this function.

In [3]:
def get_url(position, location):
    """Generate a url from position and location"""
    base_url = 'https://nl.indeed.com/jobs?q={}&l={}&start='
    url = base_url.format(position, location)
    return url

In [4]:
url = get_url('data analist', 'Nederland')

In [6]:
# check
url

'https://nl.indeed.com/jobs?q=data analist&l=Nederland&start='

In [8]:
# create driver object
driver = webdriver.Chrome('/usr/local/bin/chromedriver')
time.sleep(10)

### Access the site programmatically

- Tell the browser to visit the indeed page. Call the driver object created above and use the get method; pass the URL of the website you'd like to extract.

In [9]:
# asses webpage
driver.get(url)

- Accept cookies, if applicable.

In [10]:
try:
    driver.find_element_by_id('onetrust-accept-btn-handler').click()
    time.sleep(2)
except:
    print('probably accepted the cookie already!')

- Use BeautifulSoup; create the res object from the driver object.

In [11]:
res = driver.page_source.encode('utf-8')
soup = BeautifulSoup(res, "html.parser")

In [12]:
# store job card links in list
jobs = driver.find_elements_by_class_name('jobsearch-SerpJobCard')

# check
jobs

[<selenium.webdriver.remote.webelement.WebElement (session="d8e6f40dbe6c5e146e44595ea6762fa9", element="13c3e472-b322-43e4-a186-433cb4839780")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8e6f40dbe6c5e146e44595ea6762fa9", element="6202f0bc-40c7-4c12-a534-729f6de4c375")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8e6f40dbe6c5e146e44595ea6762fa9", element="57487880-b107-4066-92a8-68748e294a81")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8e6f40dbe6c5e146e44595ea6762fa9", element="15cda5b8-9155-4855-9414-b0caa1ef0a0f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8e6f40dbe6c5e146e44595ea6762fa9", element="ae418831-4f17-4c37-ae6e-f2ba241600d1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8e6f40dbe6c5e146e44595ea6762fa9", element="3ad8c544-5491-47d9-8711-281ae8a3c04b")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8e6f40dbe6c5e146e44595ea6762fa9", element="65a49a37-454e-418d-b575-6c

## Loop through the pages and collect the ids from the current page

In [13]:
jobids = []

# loop through first 80 pages
for i in range(0,800,10):
    driver.get(url+str(i))
    time.sleep(6)
    jobs = driver.find_elements_by_class_name('jobsearch-SerpJobCard')
    
    # collect ids from current page
    for job in jobs:
        jobid = job.get_attribute('data-jk')
        jobids.append(jobid)

In [14]:
# check first 20 ids
jobids[:20]

['024641b602f7096c',
 '54813bb6fcc05e03',
 'a9749dd1260d0ccc',
 '45f4c626b11ee6dd',
 '88d0fa9c7504ebf8',
 '3c1fa6a22de9af0a',
 '05a4124d844292f6',
 '0333f71d5aa52b3c',
 '5c387427784186c2',
 '6a28b29568a5595a',
 'fea9b49357181e96',
 '8ab7a6fcae7ca542',
 'f20e3fbd9eef4ad7',
 '2092abb8bc277a2d',
 '5abd07aaea0337b0',
 '2092abb8bc277a2d',
 'f85ea4745a4e534d',
 '4712056caf38bee1',
 '0479293efd3d04c8',
 '05225e8e90f5e755']

## Check all components separately with one job id

### Get the job title

In [25]:
driver.get('https://nl.indeed.com/vacature-bekijken?jk=024641b602f7096c')
time.sleep(5)
title = driver.find_element_by_class_name('jobsearch-JobInfoHeader-title').text

# check
title

'Data Analist'

### Get the job company

In [26]:
driver.get('https://nl.indeed.com/vacature-bekijken?jk=024641b602f7096c')
time.sleep(5)
company = driver.find_element_by_class_name('jobsearch-InlineCompanyRating').text

# check
company

'Aon'

### Get the job salary

In [27]:
driver.get('https://nl.indeed.com/vacature-bekijken?jk=024641b602f7096c')
time.sleep(5)
try:
    salary = driver.find_element_by_class_name('jobsearch-JobMetadataHeader-item').text
except:
    salary = ""

# check
salary

'€2.600 - €3.500 per maand'

### Get the job location

In [28]:
driver.get('https://nl.indeed.com/vacature-bekijken?jk=024641b602f7096c')
time.sleep(5)
location = driver.find_element_by_class_name('jobsearch-DesktopStickyContainer-subtitle').text.splitlines()[1]

# check
location

'Postcode 5651 in Eindhoven'

### Get the job company review

In [36]:
driver.get('https://nl.indeed.com/vacature-bekijken?jk=024641b602f7096c')
time.sleep(5)
companyreview = driver.find_element_by_class_name('jobsearch-CompanyReview').find_element_by_class_name('icl-Ratings-count').text

# check
companyreview

'4,610 reviews'

### Get the job description

In [37]:
driver.get('https://nl.indeed.com/vacature-bekijken?jk=024641b602f7096c')
time.sleep(5)
description = driver.find_element_by_id('jobDescriptionText').text.replace('\n',' ')

# check
description

'Over deze vacature In het kort Ben je analytisch sterk en altijd op zoek naar mogelijkheden om te helpen bij het verwezenlijken van bedrijfsdoelstellingen? Wil jij continu het beste halen uit data en met het team om tafel voor de beste strategie? Dan is de functie van Data Analist zeker iets voor jou! Jij brengt mee Minimaal hbo werk- en denkniveau Sterke affiniteit met processen en IT Grote mate van zelfstandigheid en daadkracht Je vindt bij ons Vaste 13e maand Bonusregeling Premievrij pensioen Tom von Wolzogen Kühr Recruiter 06 22581665  tom.von.wolzogen.kuhr@aon.com  linkedin.com/in/tomvonwolzogenkühr Wat ga je doen? Met de toenemende groei van data is de behoefte naar goede informatie en het gebruik en interpreteren daarvan explosief gestegen. Data is dan ook één van de internationale pijlers binnen Aon. Als Data Analist krijg je de ruimte om te pionieren. Je geeft proactief inzicht in het gedrag van onze klanten, identificeer en kwantificeer je risico’s, signaleer je trends en on

### Get the job posting date

In [38]:
driver.get('https://nl.indeed.com/vacature-bekijken?jk=024641b602f7096c')
time.sleep(5)
postingdate = driver.find_element_by_class_name('jobsearch-JobMetadataFooter').text.splitlines()[1]

# check
postingdate

'30+ dagen geleden'

# Putting it all together

- Note: if it asks you at some point to prove that you're not a robot, copy the link in a new window and select the right images.
- Tip for the the next trial: separate the ids in multiple bundles and run them separately to ensure that it doens't crush.

In [52]:
titles = []
companies = []
locations = []
salaries = []
postingdates = []
companyreviews = []
descriptions = []
scrapetime = []
#urls = []

# Use jobids2 = jobids, because you can run it for a selected segment, by setting jobids to, for example, jobids[:3] for the first 3 elements. And so, you only have to change this once, here.
jobids2 = jobids

for jobid in jobids2:
    driver.get('https://nl.indeed.com/vacature-bekijken?jk='+jobid)
    scrapetime.append(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    time.sleep(3)
    try:
        titles.append(driver.find_element_by_class_name('jobsearch-JobInfoHeader-title').text)
    except:
        titles.append('')
    try:
        companies.append(driver.find_element_by_class_name('jobsearch-InlineCompanyRating').text)
    except:
        companies.append('')
    try:
        locations.append(driver.find_element_by_class_name('jobsearch-DesktopStickyContainer-subtitle').text.splitlines()[1])
    except:
        locations.append('')
    try:
        salaries.append(driver.find_element_by_class_name('jobsearch-JobMetadataHeader-item').text)
    except:
        salaries.append('')
    try:
        postingdates.append(driver.find_element_by_class_name('jobsearch-JobMetadataFooter').text.splitlines()[1])
    except:
        postingdates.append('')
    try:
        companyreviews.append(driver.find_element_by_class_name('jobsearch-CompanyReview').find_element_by_class_name('icl-Ratings-count').text)
    except:
        companyreviews.append('')
    try:
        descriptions.append(driver.find_element_by_id('jobDescriptionText').text.replace('\n',' '))
    except:
        descriptions.append('')
    
    #To do:
    #urls.append(driver.find_element_by_class_name('###').text



### Create a data frame from the above lists

In [53]:
jobdata = pd.DataFrame(index=jobids2, data={'title':titles, 'company':companies, 'location':locations,
                                            'salary':salaries, 'postingdate':postingdates,
                                            'companyreview':companyreviews, 'description':descriptions,
                                           'scrapetime':scrapetime})

In [54]:
# check
jobdata

,title,company,location,salary,postingdate,companyreview,description,scrapetime
024641b602f7096c,Data Analist,"Aon\n4,610 reviews","4,610 reviews",€2.600 - €3.500 per maand,30+ dagen geleden,,Over deze vacature In het kort Ben je analytis...,2021-03-11 11:56:40
54813bb6fcc05e03,Data Analist,TreeHouse,Rotterdam,,Vacature rapporteren,,Pararius - het grootste online huurplatform va...,2021-03-11 11:56:43
a9749dd1260d0ccc,Traineeship: Data Analist,Createment\n24 reviews,24 reviews,"€2.500 per maand - Fulltime, Stage",8 dagen geleden,,Vraag jij je ook wel eens af hoe de apps op je...,2021-03-11 11:56:47
45f4c626b11ee6dd,Data Analist,Essent\n15 reviews,15 reviews,€3.152 - €4.902 per maand,29 dagen geleden,,Bij Essent analyseert de Data Analist CX de NP...,2021-03-11 11:56:49
88d0fa9c7504ebf8,Senior Business Analist,De Wever,Tilburg,€3.557 - €4.744 per maand,Vacature rapporteren,,Als Senior Business Analist analyseer je de im...,2021-03-11 11:56:52


### Save the data frame in a csv file

In [163]:
# save
jobdata.to_csv('data_analist_Nl_data.csv')

## To do

- If possible, tackle the Captcha